In [13]:
from pathlib import Path
import pprint

# run pip install -e . in the root directory to install this package
from stacbuilder import *

In [20]:
# Collection configuration
catalog_version = "v0.1"
collection_config_path = Path("config-collection.json")

# Input Paths
tiff_input_path = Path("/data/users/Public/emile.sonneveld/ERA5-Land-monthly-averaged-data-ANIN/tiff_collection/")
tiffs_glob = '*/*/*/*.tif'

# Output Paths
output_path = Path("results")
test_output_path = output_path / "test" / catalog_version
publish_output_path = Path("/data/users/Public/victor.verhaert/ANINStac") / catalog_version

In [15]:
# list input files
input_files = list_input_files(
    glob=tiffs_glob,
    input_dir=tiff_input_path,
    max_files=5
)
print(f"Found {len(input_files)} input files. 5 first files:")
for i in input_files[:5]: print(i) 

Found 5 input files. 5 first files:
/data/users/Public/emile.sonneveld/ERA5-Land-monthly-averaged-data-ANIN/tiff_collection/1980/01/01/1980-01-01_tp.tif
/data/users/Public/emile.sonneveld/ERA5-Land-monthly-averaged-data-ANIN/tiff_collection/1980/01/01/1980-01-01_d2m.tif
/data/users/Public/emile.sonneveld/ERA5-Land-monthly-averaged-data-ANIN/tiff_collection/1980/01/01/1980-01-01_sp.tif
/data/users/Public/emile.sonneveld/ERA5-Land-monthly-averaged-data-ANIN/tiff_collection/1980/01/01/1980-01-01_ssrd.tif
/data/users/Public/emile.sonneveld/ERA5-Land-monthly-averaged-data-ANIN/tiff_collection/1980/01/01/1980-01-01_u10.tif


In [16]:
# list meta data
asset_metadata = list_asset_metadata(
    collection_config_path=collection_config_path,
    glob=tiffs_glob,
    input_dir=tiff_input_path,
    max_files=1
)
for k in asset_metadata: 
    pprint.pprint(k.to_dict())

{'asset_id': '1980-01-01_tp',
 'asset_path': PosixPath('/data/users/Public/emile.sonneveld/ERA5-Land-monthly-averaged-data-ANIN/tiff_collection/1980/01/01/1980-01-01_tp.tif'),
 'asset_type': 'tp',
 'bbox_lat_lon': {'east': 33.05,
                  'epsg': 4326,
                  'north': -21.95,
                  'south': -35.05,
                  'west': 15.95},
 'bbox_projected': {'east': 33.05,
                    'epsg': 4326,
                    'north': -21.95,
                    'south': -35.05,
                    'west': 15.95},
 'collection_id': None,
 'datetime': datetime.datetime(1980, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
 'end_datetime': datetime.datetime(1980, 1, 31, 23, 59, 59, tzinfo=datetime.timezone.utc),
 'file_size': 135880,
 'geometry_lat_lon': <POLYGON ((15.95 -35.05, 15.95 -21.95, 33.05 -21.95, 33.05 -35.05, 15.95 -35...>,
 'href': '/data/users/Public/emile.sonneveld/ERA5-Land-monthly-averaged-data-ANIN/tiff_collection/1980/01/01/1980-01-01_tp.tif',
 'item

In [17]:
# list items
stac_items, failed_files = list_stac_items(
    collection_config_path=collection_config_path,
    glob=tiffs_glob,
    input_dir=tiff_input_path,
    max_files=10
)
print(f"Found {len(stac_items)} STAC items")
if failed_files: print(f"Failed files: {failed_files}")
if len(stac_items) > 0:
    print("First stac item:")
    pprint.pprint(stac_items[0].to_dict())

Found 10 STAC items
First stac item:
{'assets': {'tp': {'description': 'Accumulated liquid and frozen water, '
                                  'including rain and snow, that falls to the '
                                  "Earth's surface. It is the sum of "
                                  'large-scale precipitation (that '
                                  'precipitation which is generated by '
                                  'large-scale weather patterns, such as '
                                  'troughs and cold fronts) and convective '
                                  'precipitation (generated by convection '
                                  'which occurs when air at lower levels in '
                                  'the atmosphere is warmer and less dense '
                                  'than the air above, so it rises). '
                                  'Precipitation variables do not include fog, '
                                  'dew or the precipitation t

In [18]:
stac_items[0]

<Item id=1980-01-01_tp>

In [21]:
# build grouped collection
build_grouped_collections(
    collection_config_path=collection_config_path,
    glob=tiffs_glob,
    input_dir=tiff_input_path,
    output_dir=publish_output_path,
    overwrite=True,
)

In [ ]:
# build collection
build_collection(
    collection_config_path=collection_config_path,
    glob=tiffs_glob,
    input_dir=tiff_input_path,
    output_dir=test_output_path,
    overwrite=False,
)

In [3]:
# show collection
load_collection(
    collection_file=test_output_path / "collection.json"
)

<Collection id=ERA5-Land-monthly-averaged-data-v2>

In [4]:
# validate collection
validate_collection(
    collection_file=test_output_path / "collection.json",
)

In [22]:
import openeo

# Connect with VITO backend
connection = openeo.connect("https://openeo.vito.be")
connection.authenticate_oidc()

Authenticated using refresh token.


<Connection to 'https://openeo.vito.be/openeo/1.2/' with OidcBearerAuth>

In [24]:
stac_col = connection.load_stac(
    url="/data/users/Public/victor.verhaert/ANINStac/v0.1/collection.json",
    temporal_extent=["1980-01-01", "2022-12-31"],
    spatial_extent={"west": -19.95, "south": -40.05, "east": 9.95, "north": 40.05},
)

In [25]:
stac_load_job = stac_col.create_job("GTiff",
                                    title = "load stac",)
# stac_load_job.start()